<a href="https://colab.research.google.com/github/mhcayan/movie-recommendation-system-using-knn/blob/master/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
import pandas as pd
import numpy as np
import io

In [106]:
uploaded = files.upload()

Saving links.csv to links (1).csv
Saving movies.csv to movies (1).csv
Saving ratings.csv to ratings (1).csv
Saving README.txt to README (1).txt
Saving tags.csv to tags (1).csv


In [139]:
#read csv files
link_df = pd.read_csv(io.BytesIO(uploaded['links.csv']))
movie_df = pd.read_csv(io.BytesIO(uploaded['movies.csv']), index_col='movieId')
rating_df = pd.read_csv(io.BytesIO(uploaded['ratings.csv']))
tag_df = pd.read_csv(io.BytesIO(uploaded['tags.csv']))

#movie_df.isnull().any().any() #checking is there any NaN in movie_df
movie_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [140]:
def getMovieTitleById(movie_id):
  return movie_df.at[movie_id, 'title']

print(getMovieTitleById(2))


Jumanji (1995)


In [141]:
def getMovieIdByTitle(movie_title):
  return movie_df[movie_df['title'] == movie_title].index.values[0]

print(getMovieIdByTitle('Jumanji (1995)'))

2


In [142]:
rating_matrix_df = rating_df.pivot_table(index=['movieId'], columns=['userId'], values='rating')
rating_matrix_withna_df = rating_matrix_df.copy()
rating_matrix_df.fillna(0,inplace=True)
print(rating_matrix_df.shape)
rating_matrix_withna_df.head()

(9724, 610)


userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,4.5,3.5,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,5.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,2.5,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.5,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,2.5,4.0,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
#normalizing rating matrix
#rating_matrix_df = rating_matrix_df.apply(lambda row : (row - row.mean()) / (row.max() - row.min()))
rating_matrix_df.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(csr_matrix(rating_matrix_df.values))

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [152]:
def getRandomMovieId():
  return movie_df.sample().index.values[0]

random_movie_id = getRandomMovieId()
print(random_movie_id)
print(getMovieTitleById(random_movie_id))

48744
Shortbus (2006)


In [157]:
#random_movie_id = 112852

NUMBER_OF_RECOMMENDATION = 10

def getKnearestNeighbor(movie_id, K):
  return model.kneighbors(rating_matrix_df.loc[movie_id].values.reshape(1,-1), n_neighbors = K)

distances, indices = getKnearestNeighbor(random_movie_id,NUMBER_OF_RECOMMENDATION)

distances = distances.flatten()
indices = indices.flatten()

print("Recommended movie for : %6d %50s\n" % (random_movie_id, getMovieTitleById(random_movie_id)))

print("----------------------------------------------------------------------------------")
print("%6s | %65s | %15s\n" % ("Id", "Title", "similarity_score"))
print("----------------------------------------------------------------------------------")
for i in range(1,len(distances)):
    movieId = rating_matrix_df.iloc[indices[i]].name
    print("%6d | %65s | %2.9f\n" % (movieId, getMovieTitleById(movieId), 1- distances[i]))



Recommended movie for :  48744                                    Shortbus (2006)

----------------------------------------------------------------------------------
    Id |                                                             Title | similarity_score

----------------------------------------------------------------------------------
  5613 |                                                    8 Women (2002) | 0.647580478

 34164 |                                              Happy Endings (2005) | 0.646162343

  7122 |                                             King of Hearts (1966) | 0.646162343

 58492 |                                                Snow Angels (2007) | 0.646162343

 40955 |                                         Breakfast on Pluto (2005) | 0.591971207

111384 |                                                  Blue Ruin (2013) | 0.574366527

  6119 |     Santa Claus Is a Stinker (Le Père Noël est une ordure) (1982) | 0.574366527

 96945 |            Love L

In [164]:
#return list of user rated movies(id) along with rating value. user's mean rating is subtracted from actual rating

GLOBAL_USER_BIAS = 0.001 #this bias is added for user's who have same rating for all the movies, which will result in zero score for all his recommeded movies
def getUserRatedMovies(userId):
  userMeanRating = rating_matrix_withna_df[userId].mean()
  userRatedMovies = rating_matrix_withna_df[rating_matrix_withna_df[userId].notnull()][userId].subtract(userMeanRating-GLOBAL_USER_BIAS)
  return list(zip(userRatedMovies.index, userRatedMovies)) 

print(getUserRatedMovies(2))



[(318, -0.9472758620689654), (333, 0.05272413793103459), (1704, 0.5527241379310346), (3578, 0.05272413793103459), (6874, 0.05272413793103459), (8798, -0.4472758620689654), (46970, 0.05272413793103459), (48516, 0.05272413793103459), (58559, 0.5527241379310346), (60756, 1.0527241379310346), (68157, 0.5527241379310346), (71535, -0.9472758620689654), (74458, 0.05272413793103459), (77455, -0.9472758620689654), (79132, 0.05272413793103459), (80489, 0.5527241379310346), (80906, 1.0527241379310346), (86345, 0.05272413793103459), (89774, 1.0527241379310346), (91529, -0.4472758620689654), (91658, -1.4472758620689654), (99114, -0.4472758620689654), (106782, 1.0527241379310346), (109487, -0.9472758620689654), (112552, 0.05272413793103459), (114060, -1.9472758620689654), (115713, -0.4472758620689654), (122882, 1.0527241379310346), (131724, 1.0527241379310346)]


In [166]:
#Computing popularity df, which will contain ratingCount, meanRating, and popularity score for each movie

global_mean_rating = rating_df['rating'].mean()
popularity_df = rating_df.groupby(by=['movieId'])['rating'].count().reset_index().rename(columns={'rating':'ratingCount'})
popularity_df.head()
popularity_df['meanRating'] = rating_df.groupby(by=['movieId'])['rating'].mean().reset_index().rename(columns={'rating':'ratingMean'})['ratingMean']
popularity_df.head()
m = popularity_df['ratingCount'].quantile(0.7) 

#Formula adopted from: http://trailerpark.weebly.com/imdb-rating.html?source=post_page

popularity_df['weightedRating'] = ((popularity_df['meanRating'] * popularity_df['ratingCount'] + global_mean_rating * m) / 
                                     (popularity_df['ratingCount'] + m))

popularity_df = popularity_df.merge(movie_df[['title']], how = 'left', on='movieId') #adding the movie title for display purpose

popularity_df.sort_values(['weightedRating', 'meanRating'], ascending=[False, False], inplace = True)
popularity_df.head()


,movieId,ratingCount,meanRating,weightedRating,title
277,318,317,4.429022,4.408984,"Shawshank Redemption, The (1994)"
659,858,192,4.289062,4.261361,"Godfather, The (1972)"
2224,2959,218,4.272936,4.248937,Fight Club (1999)
840,1104,20,4.475000,4.222626,"Streetcar Named Desire, A (1951)"
921,1221,129,4.259690,4.220668,"Godfather: Part II, The (1974)"


In [167]:
#Returns list of top N popular movies based on weighted rating
#Each list element is a tuple of (movieId, title, weightedRating)
def getPopularMovies(N):
  return [tuple(x) for x in popularity_df.iloc[:N][['movieId', 'title', 'weightedRating']].to_numpy()]

print(getPopularMovies(5))

[(318, 'Shawshank Redemption, The (1994)', 4.408984255818885), (858, 'Godfather, The (1972)', 4.26136130093125), (2959, 'Fight Club (1999)', 4.248937328379195), (1104, 'Streetcar Named Desire, A (1951)', 4.222625884641435), (1221, 'Godfather: Part II, The (1974)', 4.220668374156755)]


In [0]:
#This method will be used for cold start users, 50% of his recommeded movies will come from popular movies
#param: userRatedMovies - set containing already rated movies by the user
#param: recommendedList - list of recommeded movies found using KNN. popular movies will be added to this list

def addPopularMoviesToRecommendationList(userRatedMovies, recommendedList):
  
  popularMovies = getPopularMovies(len(userRatedMovies) + NUMBER_OF_RECOMMENDATION)

  #set of already recommeded movies. Will be used to avoid a popular movie that is already recommended
  recommendedMovieSet = set([x[0] for x in recommendedList])
  #adding popular movies to recommendation list until we get required number of recommendation
  for movie in popularMovies:
    
    #avoid adding movies that are already in the recommedation list or already rated by user
    if (movie[0] not in userRatedMovies) and (movie[0] not in recommendedMovieSet):
      recommendedList.append(movie)
      if(len(recommendedList) >= NUMBER_OF_RECOMMENDATION):
        break
#l = []
#addPopularMoviesToRecommendationList(set(), l)
#print(l)

In [169]:
def getRandomUserId():
  return rating_df.sample()['userId'].values[0]

print(getRandomUserId())

105


In [0]:
COLD_USER_THRESHOLD = 50
def isColdUser(userRatedMovies:list):
  return len(userRatedMovies) < COLD_USER_THRESHOLD

In [183]:
#Returns list of recommeded movies for given userId

def getRecommendationForUser(userId):
  
  userRatedMovies = getUserRatedMovies(userId)
  ratedMovieSet = set()
  for item in userRatedMovies:
    ratedMovieSet.add(item[0])

  recommendationDict = dict()

  for movieId, rating in userRatedMovies:
    distances, indices = getKnearestNeighbor(movieId, NUMBER_OF_RECOMMENDATION)
    distances = distances.flatten()
    indices = indices.flatten()
    for i in range(1,len(distances)):
      neighborId = rating_matrix_df.iloc[indices[i]].name
      neighborSimilarity = 1 - distances[i]
      neighborScore = neighborSimilarity * rating

      if neighborId not in ratedMovieSet:
        recommendationDict[neighborId] = recommendationDict.get(neighborId, 0) + neighborScore

  recommendationList = [(k,v) for k,v in sorted(recommendationDict.items(), key = lambda item : item[1], reverse = True)]

  #For cold start user 50% of the recommended movies will come from popular movies
  if isColdUser(userRatedMovies):
    recommendationList = recommendationList[:int(NUMBER_OF_RECOMMENDATION/2)] #remove 50% movie from recommendation list
    addPopularMoviesToRecommendationList(ratedMovieSet, recommendationList)
  elif len(recommendationList) < NUMBER_OF_RECOMMENDATION:
    addPopularMoviesToRecommendationList(ratedMovieSet, recommendationList)
  
  return recommendationList[:NUMBER_OF_RECOMMENDATION]
  
def testGetRecommendationForUser():
  
  userId = getRandomUserId()
  print("Recommended movie for user: %6d\n" % (userId))

  print("------------------------------------------------------------------------------------------------")
  print("%6s | %66s | %10s  | %5s\n" % ("ID", "TITLE", "SCORE", "FROM_POPULAR_LIST"))
  print("------------------------------------------------------------------------------------------------")
  recommendationList = getRecommendationForUser(userId)
  for movie in recommendationList:
    if len(movie) == 2:
      movieId = movie[0]
      score = movie[1]
      print("%6d | %66s | %2.9f | NO\n" % (movieId, getMovieTitleById(movieId), score))
    else:
      movieId = movie[0]
      title = movie[1]
      weightedRating = movie[2]
      print("%6d | %66s | %2.9f | YES\n" % (movieId, title, weightedRating))

testGetRecommendationForUser()


Recommended movie for user:    210

------------------------------------------------------------------------------------------------
    ID |                                                              TITLE |      SCORE  | FROM_POPULAR_LIST

------------------------------------------------------------------------------------------------
 88140 |                          Captain America: The First Avenger (2011) | 2.102310888 | NO

  1291 |                          Indiana Jones and the Last Crusade (1989) | 2.091629047 | NO

 68954 |                                                          Up (2009) | 2.033893044 | NO

  1240 |                                             Terminator, The (1984) | 1.798553119 | NO

  3793 |                                                       X-Men (2000) | 1.599287552 | NO

  5445 |                                             Minority Report (2002) | 1.528822723 | NO

  6333 |                                            X2: X-Men United (2003) | 1.359